# Homework Assignment  5

**Question 1:** Select a directed network, and develop a simple
    link prediction classifier based on the network's directed topology (40pt). 
    Use node or link embedding features and try to improve the classification (20pt)
    
**Bonus:** Evaluate the classifier's ability to predict future links (10pt)
    
** Note:** You can use [Reddit community networks](http://dynamics.cs.washington.edu/data.html), or search
for networks using [The Colorado Index of Complex Networks](https://icon.colorado.edu/).

In [2]:
import pandas as pd
from google.colab import drive
!pip install turicreate
!pip install networkx
drive.mount("/content/gdrive", force_remount=True)
import turicreate as tc 
import networkx as nx 


Mounted at /content/gdrive


In [3]:
data=pd.read_csv("/content/gdrive/MyDrive/Studying/Big Data/HW5_203837695/requirements.csv")
#data= tc.SFrame.read_csv("/content/gdrive/MyDrive/Studying/Big Data/HW5_203837695/requirements.csv")
len(data)

110104

In [8]:
data.iloc[:5]

,package,requirement,package_name,package_version
0,02exercicio-1.0.0,NaN,02exercicio,1.0.0
1,0x10c-asm-0.0.2,NaN,0x10c-asm,0.0.2
2,115wangpan-0.7.6,beautifulsoup4,115wangpan,0.7.6
3,115wangpan-0.7.6,homura,115wangpan,0.7.6
4,115wangpan-0.7.6,humanize,115wangpan,0.7.6


In [9]:
#create nodes
p=set(data["package_name"])
r=set(data['requirement'])
nodes=list(p |r)
#create edges
data["requirement"]=data["requirement"].astype(str)
edges=[(a["package_name"], a["requirement"]) for i,a in data.iterrows() if a["requirement"]!="nan"]
edges[:5]

[('115wangpan', 'beautifulsoup4'),
 ('115wangpan', 'homura'),
 ('115wangpan', 'humanize'),
 ('115wangpan', 'pycurl'),
 ('115wangpan', 'pysocks')]

In [10]:
g=nx.DiGraph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
print(nx.info(g))

DiGraph with 58742 nodes and 72641 edges


In [11]:
import random 

In [12]:
# This is for predicting missing links scenario

def select_random_existing_links(g, n):
    l =list(g.edges())
    random.shuffle(l)
    return l[:n]

def select_random_non_existing_links(g, n):
    select_links = set()
    v_list = list(g.nodes())
    while len(select_links) < n:
        u = random.choice(v_list)
        v = random.choice(v_list)
        if g.has_edge(u,v) or (u,v) in select_links and u == v:
            continue
        else:
            select_links.add((u,v))
    return list(select_links)
            
    
postive_links = select_random_existing_links(g,10000)
negative_links = select_random_non_existing_links(g,10000)

In [13]:
!pip install tqdm
from tqdm import tqdm_notebook as tqdm 

In [14]:
def find_friends_of_friends(g, v):
    "Returns a set with all the friends of friends of v that are not friends of v"
    friends_set = set(g.neighbors(v))
    friends_of_friends_set = set()
    for f in friends_set:
        friends_of_friends_set |= set(g.neighbors(f))
    friends_of_friends_set -= friends_set
    friends_of_friends_set -= {v}
    return friends_of_friends_set

def rank_friends_of_friends(g, v):
    rank_dict = {}
    vertices_set = find_friends_of_friends(g, v)
    for u in vertices_set:
        rank_dict[u] = len(common_friends(g, v, u))
    sorted_d = sorted(rank_dict.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_d

def common_friends(g,u,v):
    if not g.has_node(u) or not g.has_node(v):
        return None
    return set(g.neighbors(u)) & set(g.neighbors(v))

def friends_measure(g, u,v):
    s = 0
    for x in find_friends_of_friends(g,u):
        for y in find_friends_of_friends(g,v):
            if g.has_edge(x,y) or x == y:
                s += 1
    return s

def total_friends(g, u, v):   
    return len(set(g.neighbors(v)) | set(g.neighbors(u)))

def get_features_dict(g,u, v):
    removed_edge = False
    if g.has_edge(u,v):
        g.remove_edge(u,v) #  remove edge before feature calculation
        removed_edge = True
    try:
        d = {
            'degree_v': len(list(g.neighbors(v))),
            'degree_u': len(list(g.neighbors(u))),
            'total_friends': total_friends(g,u,v),
            'common_friends': len(common_friends(g,u,v)),
            'friends_measure': friends_measure(g, u,v)
        }
    except:
        if removed_edge:
            g.add_edge(u,v) # return the edge back to the graph
        return None

    
    if removed_edge:
        g.add_edge(u,v) # return the edge back to the graph
    return d




In [15]:
links_features = []

for i in tqdm(range(len(postive_links))):    
    u,v = postive_links[i]
    d = get_features_dict(g, u,v )
    if d is None:
        continue
    d['class'] = True
    links_features.append(d)

for i in tqdm(range(len(negative_links))):    
    u,v = negative_links[i]
    d = get_features_dict(g, u,v )
    if d is None:
        continue
    d['class'] = False
    links_features.append(d)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/10000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/10000 [00:00<?, ?it/s]

In [16]:
links_features[0]

{'class': True,
 'common_friends': 0,
 'degree_u': 3,
 'degree_v': 0,
 'friends_measure': 0,
 'total_friends': 3}

In [17]:
features_list = ['degree_v', 'degree_u', 'total_friends', 'common_friends', 'friends_measure', 'class' ]
data_dict = {f:[] for f in features_list}
 
for d in links_features:
    for feature in features_list:
        data_dict[feature].append(d[feature])
sf = tc.SFrame(data_dict)
sf
    

class,common_friends,degree_u,degree_v,friends_measure,total_friends
1,0,3,0,0,3
1,0,12,0,0,12
1,1,6,1,0,6
1,0,8,0,0,8
1,0,5,2,0,7
1,0,6,0,0,6
1,0,4,0,0,4
1,0,2,0,0,2
1,0,4,1,0,5
1,2,2,2,3,2


In [18]:
test, train  = sf.random_split(0.2)
print(f"Train size {len(train)}\nTest size: {len(test)}" )
features_list = ['degree_v', 'degree_u', 'total_friends', 'common_friends', 'friends_measure' ]
cls = tc.classifier.create(train,features=features_list, target="class")


Train size 15812
Test size: 4188
PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: BoostedTreesClassifier, RandomForestClassifier, DecisionTreeClassifier, SVMClassifier, LogisticClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.


Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 15021

Number of classes           : 2

...

| 5         | 6        | 0.128142     | 0.748352          | 0.709229            |

| 8         | 9        | 0.174447     | 0.748485          | 0.709229            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

PROGRESS: Model selection based on validation accuracy:
PROGRESS: ---------------------------------------------
PROGRESS: BoostedTreesClassifier          : 0.7914032869785083
PROGRESS: RandomForestClassifier          : 0.7914032869785083
PROGRESS: DecisionTreeClassifier          : 0.7850821744627055
PROGRESS: SVMClassifier                   : 0.706700379266751
PROGRESS: LogisticClassifier              : 0.7092288242730721
PROGRESS: ---------------------------------------------
PROGRESS: Selecting BoostedTreesClassifier based on validation set performance.


In [19]:
results = cls.evaluate(test)
results

{'accuracy': 0.7977554918815664,
 'auc': 0.8728649176778992,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  298  |
 |      1       |        1        |  1809 |
 |      0       |        1        |  549  |
 |      0       |        0        |  1532 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.8103023516237403,
 'log_loss': 0.4417084643599894,
 'precision': 0.767175572519084,
 'recall': 0.8585666824869482,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+-----+-----+------+------+
 | threshold | fpr | tpr |  p   |  n   |
 +-----------+-----+-----+------+------+
 |    0.0    | 1.0 | 1.0 | 2107 | 2081 |
 |   0.001   | 1.0 | 1.0 | 2107 | 2081 |
 |   0.002   | 1.0 | 1

## Node Embedding

In [20]:
graph_num = nx.convert_node_labels_to_integers(g, first_label=0, ordering='default')

In [21]:
!pip install karateclub
import karateclub

In [22]:
graph_num = nx.convert_node_labels_to_integers(g, first_label=0, ordering='default')

In [23]:
model = karateclub.DeepWalk()
model.fit(graph_num)
embedding = model.get_embedding()
embedding

array([[ 2.1810023e-02,  3.0888405e-02,  5.3695343e-03, ...,
         1.4912099e-02, -2.3383059e-02, -2.1115202e-03],
       [-8.8259327e-01, -1.1166120e+00, -9.2517215e-01, ...,
        -2.8098276e-01,  1.8952339e+00, -1.3460118e-01],
       [-3.3706151e-02, -2.0392409e-01,  3.9017284e-01, ...,
        -2.8584373e-01,  2.5796884e-01, -3.2449748e-02],
       ...,
       [-9.4102943e-01, -8.7142251e-02, -9.6809119e-01, ...,
        -4.5557716e-01,  1.0975548e+00,  1.5670617e-01],
       [-1.6777219e-03,  1.2231249e-03,  4.9781897e-03, ...,
         6.2826057e-03,  7.9138315e-04,  1.3062265e-02],
       [ 4.4327188e-02,  2.5172593e-02,  4.6750572e-02, ...,
        -8.5528180e-02,  1.5462892e-01,  1.9539396e-01]], dtype=float32)

In [24]:
pos=pd.DataFrame(embedding)
pos["class"]=1
pos_sample=pos.sample(1000)

In [25]:
import numpy as np
neg=pd.DataFrame(np.random.randn(len(pos),len(pos.columns)-1), columns=np.arange(len(pos.columns)-1))
neg["class"]=0
neg_sample=neg.sample(1000)

In [26]:
data=pos_sample.append(neg_sample).sample(len(pos_sample)+len(neg_sample))
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,class
4035,-0.116811,0.031463,-0.011253,0.031894,0.285853,0.055026,0.271400,-0.487831,0.181252,0.108335,0.355117,-0.080733,-0.392700,-0.258600,0.365439,-0.098561,-0.051523,0.191249,-0.285011,0.007136,-0.086075,0.096262,0.426751,-0.400569,-0.232444,0.403440,-0.365035,-0.118223,-0.342807,-0.363037,-0.021078,0.368762,0.417412,-0.260742,0.029657,0.119617,0.025952,0.267090,0.011858,0.163387,...,-0.042755,0.134511,0.393841,0.041545,-0.055562,-0.224673,0.199797,0.241799,0.253839,0.040318,0.548881,-0.355127,-0.408507,-0.064497,-0.025113,0.175457,-0.009414,0.264313,-0.169938,0.022099,-0.138034,-0.353278,0.025148,0.211552,0.331581,0.080749,-0.060902,-0.278013,0.100691,0.038886,0.124432,-0.406500,0.084227,0.013685,0.085890,-0.293952,0.120315,0.162751,0.275380,1
51134,0.055134,-0.112355,0.049640,-0.057208,0.050564,-0.079221,-0.016464,-0.135587,-0.120093,0.015653,-0.009428,-0.074894,-0.018458,-0.056054,0.020133,0.001287,-0.191004,-0.004845,0.092934,-0.165953,0.081604,-0.023096,-0.003312,-0.144177,0.019064,0.033186,0.098876,0.039932,-0.106666,0.034382,-0.173731,-0.145832,-0.003178,0.062536,0.036987,-0.062900,-0.042350,0.030422,0.020718,0.027900,...,0.129774,0.067549,0.142275,0.105847,-0.061842,0.058369,0.031327,0.072698,-0.020891,-0.056818,-0.126216,0.006440,-0.122530,-0.032520,-0.029710,-0.120647,0.153410,-0.030196,-0.057200,0.115210,0.107932,-0.150575,-0.106910,0.086371,0.076503,-0.005198,-0.149858,-0.124122,0.135255,0.166883,0.018057,0.011601,0.021071,-0.052639,-0.064510,-0.126348,-0.060390,0.119078,0.043648,1
37189,-1.005575,-0.168372,-0.204143,-0.077094,0.602026,-0.808043,1.371450,1.302968,-0.171419,0.394515,0.611225,0.911845,1.045622,0.779511,0.401713,0.065666,0.022622,-0.835058,-0.289717,-0.515457,0.643010,-0.338046,0.464223,-0.089464,0.114537,-0.444918,0.015886,0.543948,0.032425,-0.310397,-0.591524,0.548351,-0.074061,0.605489,-0.568378,-0.256296,-0.431886,0.896789,0.669331,0.529369,...,-1.014650,0.195360,0.460142,0.354450,-0.082867,0.566054,-0.990264,0.917173,0.593197,0.033668,-0.042572,0.052885,0.070844,-0.451989,-0.322286,0.047899,0.939218,0.106422,-0.090419,0.555499,-0.760392,-0.402677,-0.034044,-0.068428,0.808531,-0.184166,-0.344680,-1.613281,1.200199,0.378502,0.441582,-0.601093,1.525199,-1.066618,-0.504960,0.221571,-0.482595,0.869624,0.620085,1
47896,-0.690051,0.219422,-0.925691,0.246370,0.497450,-1.562762,-0.233300,0.551026,0.655368,0.400981,0.520383,0.726187,0.825904,0.116325,-0.110088,-0.606927,-0.300260,-0.215130,-0.718568,1.036493,-0.893616,0.531817,0.009397,-0.725607,0.458659,0.532538,0.231830,0.209092,-1.121237,0.124536,-0.602800,0.340393,0.310920,-0.427498,0.475375,-0.238879,-0.242085,0.722431,0.518464,0.750886,...,-0.459934,0.247196,-0.240905,-0.029175,-0.843135,-0.664033,-0.136728,-0.021241,0.535440,-0.104537,0.616367,-0.332107,0.518907,-0.679054,-0.065921,0.073760,0.530789,0.364317,1.496720,-0.189827,0.567208,-0.435161,-0.254152,-0.140521,1.682769,0.720207,0.600700,-0.465050,0.424097,0.263361,0.289817,-0.734249,0.442991,-0.173739,0.499284,-1.052059,0.023768,0.406461,-0.840650,1
5246,0.133856,-1.345990,-0.000816,0.397146,1.491067,-0.153364,-0.449028,0.011806,-0.439634,1.133394,0.503423,-0.194356,0.283719,-0.400301,-0.216567,-0.741980,0.103207,-0.137192,-1.070469,0.060794,-0.781252,0.864661,-0.746889,-0.652261,0.452749,0.517683,-0.385305,-0.152235,0.332986,-1.023638,-1.008547,0.359732,1.644131,-1.881701,-0.116496,0.846789,-0.119308,0.668282,0.627069,1.389538,...,0.242331,-0.728734,0.938666,0.973283,-1.705934,0.560356,-0.837028,0.296464,0.307585,1.157334,1.391468,-0.775402,-0.381324,0.014824,-0.890762,-0.906026,-0.073566,0.149509,0.193731,-0.335604,-0.562933,0.348019,-0.257231,0.398103,0.387934,1.111623,0.495669,-1.393397,0.084003,-0.562892,0.405151,-0.597418,-0.40

In [27]:
e=tc.SFrame(data=data)
test, train  = e.random_split(0.2)
print(f"Train size {len(train)}\nTest size: {len(test)}" )
cls = tc.classifier.create(train,features=[str(a) for a in data.columns[:-1].values], target="class")

Train size 1611
Test size: 389
PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: BoostedTreesClassifier, RandomForestClassifier, DecisionTreeClassifier, SVMClassifier, LogisticClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.


Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 1530

Number of classes           : 2

...

| 5         | 6        | 0.332822     | 0.784967          | 0.777778            |

| 6         | 7        | 0.385165     | 0.784967          | 0.777778            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

PROGRESS: Model selection based on validation accuracy:
PROGRESS: ---------------------------------------------
PROGRESS: BoostedTreesClassifier          : 1.0
PROGRESS: RandomForestClassifier          : 0.9753086419753086
PROGRESS: DecisionTreeClassifier          : 0.9382716049382716
PROGRESS: SVMClassifier                   : 0.7654320987654321
PROGRESS: LogisticClassifier              : 0.7777777777777778
PROGRESS: ---------------------------------------------
PROGRESS: Selecting BoostedTreesClassifier based on validation set performance.


In [28]:
results = cls.evaluate(test)
results

{'accuracy': 0.987146529562982,
 'auc': 0.9993368348453497,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |   2   |
 |      0       |        1        |   3   |
 |      0       |        0        |  203  |
 |      1       |        1        |  181  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9863760217983651,
 'log_loss': 0.09263990188663396,
 'precision': 0.9836956521739131,
 'recall': 0.9890710382513661,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+-----+-----+-----+-----+
 | threshold | fpr | tpr |  p  |  n  |
 +-----------+-----+-----+-----+-----+
 |    0.0    | 1.0 | 1.0 | 183 | 206 |
 |   0.001   | 1.0 | 1.0 | 183 | 206 |
 |   0.002   | 1.0 | 1.0 | 183 

# Question 2

**Question 2:** Use three different centrality algorithms to identify managers at Enron.
    Evaluate the precision@10 of the algorithms, i.e. how many managers attained the top-10 highest ranks (30pt). Draw 
    the network and visualize one of the centrality measures (10pt)
    

In [29]:
#setting up Kaggle & TuriCreate package s
import json
import os

!mkdir /root/.kaggle/
# Installing the Kaggle package
!pip install kaggle 

#Important Note: complete this with your own key - after running this for the first time remmember to **remove** your API_KEY
api_token ={"username":"amirbialer","key":"f2af62d43f0aaf0ee3bea5998634a1b7"}

# creating kaggle.json file with the personal API-Key details 
# You can also put this file on your Google Drive

with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

In [30]:
!mkdir ./datasets
!mkdir ./datasets/enron
!wget -O ./datasets/enron/enron_mail_20150507.tar.gz https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tar.gz
!tar -zxvf ./datasets/enron/enron_mail_20150507.tar.gz -C ./datasets/enron/
!ls ./datasets/enron

Streaming output truncated to the last 5000 lines.
maildir/campbell-l/all_documents/1572.
maildir/campbell-l/all_documents/759.
maildir/campbell-l/all_documents/538.
maildir/campbell-l/all_documents/876.
maildir/campbell-l/all_documents/321.
maildir/campbell-l/all_documents/990.
maildir/campbell-l/all_documents/1842.
maildir/campbell-l/all_documents/1460.
maildir/campbell-l/all_documents/1840.
maildir/campbell-l/all_documents/274.
maildir/campbell-l/all_documents/448.
maildir/campbell-l/all_documents/1692.
maildir/campbell-l/all_documents/400.
maildir/campbell-l/all_documents/224.
maildir/campbell-l/all_documents/188.
maildir/campbell-l/all_documents/1670.
maildir/campbell-l/all_documents/1678.
maildir/campbell-l/all_documents/706.
maildir/campbell-l/all_documents/254.
maildir/campbell-l/all_documents/1429.
maildir/campbell-l/all_documents/1704.
maildir/campbell-l/all_documents/529.
maildir/campbell-l/all_documents/1039.
maildir/campbell-l/all_documents/1694.
maildir/campbell-l/all_doc

In [31]:
import os, re
from glob import glob
from tqdm import tqdm_notebook as tqdm 

enron_data_set_path = "./datasets/enron/maildir"

def get_links_from_email(path):
    email_re = re.compile(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)")
    to_list = []
    from_list = []
    for line in open(path, "r", encoding='utf8', errors='ignore').readlines():
        line = line.strip().lower()
        if line.startswith("to:"):
            to_list = email_re.findall(line)
        if line.startswith("from:"):
            from_list = email_re.findall(line)
    links = set()
    if len(from_list) > 0 and len(to_list) >0:
        for e1 in from_list:
            for e2 in to_list:
                links.add((e1,e2))
    return links          

links = set()
files_list = [p for p in glob(f"{enron_data_set_path}/*/*/*") if "/inbox/" in p or "/sent/" in p]
print("Found %s files" % len(files_list))
for i in tqdm(range(len(files_list))):
    p = files_list[i]              
    if os.path.isfile(p):
        try:
            links |= get_links_from_email(p)
        except:
            continue

g = nx.DiGraph()   

Found 99279 files


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/99279 [00:00<?, ?it/s]

In [32]:
g = nx.DiGraph()
g.add_edges_from(links)
print(nx.info(g))

DiGraph with 18590 nodes and 32392 edges


In [33]:
f = open("/content/gdrive/MyDrive/Studying/Big Data/HW5_203837695/roles.txt", "r")
l=f.readlines()

In [34]:
g.nodes

NodeView(('kay.mann@enron.com', 'ann.davis@swpc.siemens.com', 'doyle@rff.org', 'william.nordhaus@yale.edu', 'wiegert@internetni.com', 'falancer.mary@enron.com', 'jeff.dasovich@enron.com', 'michaelberg@icfconsulting.com', 'eric.bass@enron.com', 'gspajda@acsu.buffalo.edu', 'kkinsella@mail.ccsd.k12.co.us', 'timkenney@yahoo.com', 'gerald.nemec@enron.com', 'julia.murray@enron.com', 'wicain@fame.com', 'tdonoho@ect.enron.com', 'jay.webb@enron.com', 'mark.pickering@enron.com', 'jeffrey.miller@enron.com', 'phillip.platter@enron.com', 'scarey@isda.org', 'fhampton@bondmarkets.com', 'sholmes@nicor.com', 'pmims@enron.com', 'bethc@pandaenergy.com', '.ward@enron.com', 'bvirtue@cogentech.com', 'rmcnair@cogentech.com', 'danny.clark@enron.com', 'russ.kavanaugh@enron.com', 'ayesha.kanji@enron.com', 'fp.dl-warriors@enron.com', 'jeanette.doll@enron.com', 'blair.lichtenwalter@enron.com', 'richard.melton@enron.com', 'emily.sellers@enron.com', 'andy.blanchard@enron.com', 'kenneth.lay@enron.com', 'mike.grigsby

In [40]:
import re
managers=pd.Sries([re.match("(.*?)(\t)", w)[0][:-2] for w in l if ("manag" in w.lower() or "director" in w.lower() or "president" in w.lower() or "ceo" in w.lower())])


In [43]:
df = pd.read_table("http://www.inf.ed.ac.uk/teaching/courses/tts/assessed/roles.txt", error_bad_lines=False, names=["name1", "name2"])
manager_list=["president", "ceo", "manag"]
df['is_manager'] = [True if any(m in name.lower() for m in manager_list) else False for name in df['name2']]
df["is_manager"]

0      False
1       True
2      False
3      False
4       True
       ...  
180    False
181    False
182     True
183    False
184    False
Name: is_manager, Length: 185, dtype: bool

In [44]:
manager_names=df[df["is_manager"]]["name1"]

In [45]:
d1 = nx.closeness_centrality(g)

d2 = nx.voterank(g)

d3 = nx.degree_centrality(g)


In [48]:
closeness_centrality=pd.Series(d1).sort_values(ascending=False)[:10]
voterank=pd.Series(d2).sort_values(ascending=False)[:10]
degree_centrality=pd.Series(d3).sort_values(ascending=False)[:10]

In [54]:
def Calc_Prec10(top_central):
  c=0
  for central in top_central:
    for m in manager_names:
      if m in central:
        c+=1
  return c


In [63]:
closeness=Calc_Prec10(closeness_centrality.keys().values)
vote=Calc_Prec10(voterank.values)
degree=Calc_Prec10(degree_centrality.keys().values)

In [64]:
print("closeness_centrality precision@10: {}%".format(closeness*10))
print("voterank precision@10: {}%".format(vote*10))
print("degree_centrality precision@10: {}%".format(degree*10))

closeness_centrality precision@10: 80%
voterank precision@10: 0%
degree_centrality precision@10: 30%


# Draw

In [67]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,20))
nx.draw_spring(g, with_labels=True, node_size=[v * 100 for v in g.nodes.values()])

TypeError: ignored

<Figure size 1440x1440 with 0 Axes>

In [68]:
g.nodes.values()

ValuesView(NodeView(('kay.mann@enron.com', 'ann.davis@swpc.siemens.com', 'doyle@rff.org', 'william.nordhaus@yale.edu', 'wiegert@internetni.com', 'falancer.mary@enron.com', 'jeff.dasovich@enron.com', 'michaelberg@icfconsulting.com', 'eric.bass@enron.com', 'gspajda@acsu.buffalo.edu', 'kkinsella@mail.ccsd.k12.co.us', 'timkenney@yahoo.com', 'gerald.nemec@enron.com', 'julia.murray@enron.com', 'wicain@fame.com', 'tdonoho@ect.enron.com', 'jay.webb@enron.com', 'mark.pickering@enron.com', 'jeffrey.miller@enron.com', 'phillip.platter@enron.com', 'scarey@isda.org', 'fhampton@bondmarkets.com', 'sholmes@nicor.com', 'pmims@enron.com', 'bethc@pandaenergy.com', '.ward@enron.com', 'bvirtue@cogentech.com', 'rmcnair@cogentech.com', 'danny.clark@enron.com', 'russ.kavanaugh@enron.com', 'ayesha.kanji@enron.com', 'fp.dl-warriors@enron.com', 'jeanette.doll@enron.com', 'blair.lichtenwalter@enron.com', 'richard.melton@enron.com', 'emily.sellers@enron.com', 'andy.blanchard@enron.com', 'kenneth.lay@enron.com', 'm